In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StructType,StructField,IntegerType,FloatType,LongType,StringType
from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.dataset.spark_splitters import spark_random_split
from pyspark.sql.functions import regexp_replace, col


In [2]:
sc = pyspark.SparkContext()
sql = SQLContext(sc)

In [3]:
spark = SparkSession \
    .builder \
    .appName("ALS") \
    .getOrCreate()

In [4]:
spark.version

'3.1.1'

In [5]:
df = spark.read.csv('BX-Book-Ratings.csv',header=True
                   ,sep=';')


In [6]:
df.show(10)

+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
| 276727|0446520802|          0|
| 276729|052165615X|          3|
| 276729|0521795028|          6|
| 276733|2080674722|          0|
| 276736|3257224281|          8|
| 276737|0600570967|          6|
| 276744|038550120X|          7|
| 276745| 342310538|         10|
+-------+----------+-----------+
only showing top 10 rows



### Remove Alphabet from aphanumeric ISBN Columns using 'withColumn' function

In [7]:
df=df.withColumn("ISBN",regexp_replace(col("ISBN"),"[a-zA-Z]",""))


In [8]:
COL_USER = "User-ID"
COL_ITEM = "ISBN"
COL_RATING = "Book-Rating"
COL_PREDICTION = "prediction"

schema=StructType(
    (
    StructField(COL_USER,LongType()),
    StructField(COL_ITEM,LongType()),
    StructField(COL_RATING,FloatType())))





In [9]:
df.first()

Row(User-ID='276725', ISBN='034545104', Book-Rating='0')

<h2>Changing the schema of the Dataframe using 'withColumn' method</h2>

In [10]:
df2 = df.withColumn("User-ID",col("User-ID").cast(IntegerType())) \
    .withColumn("ISBN",col("ISBN").cast(IntegerType())) \
    .withColumn("Book-Rating",col("Book-Rating").cast(FloatType()))

In [11]:
df.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)



In [12]:
df2.printSchema()

root
 |-- User-ID: integer (nullable = true)
 |-- ISBN: integer (nullable = true)
 |-- Book-Rating: float (nullable = true)



In [13]:
df2.select('ISBN').show(5)

+---------+
|     ISBN|
+---------+
| 34545104|
|155061224|
|446520802|
| 52165615|
|521795028|
+---------+
only showing top 5 rows



In [14]:
df2.show(4)

+-------+---------+-----------+
|User-ID|     ISBN|Book-Rating|
+-------+---------+-----------+
| 276725| 34545104|        0.0|
| 276726|155061224|        5.0|
| 276727|446520802|        0.0|
| 276729| 52165615|        3.0|
+-------+---------+-----------+
only showing top 4 rows



In [15]:
df2.describe().show()

+-------+------------------+--------------------+------------------+
|summary|           User-ID|                ISBN|       Book-Rating|
+-------+------------------+--------------------+------------------+
|  count|           1149780|             1062289|           1149780|
|   mean|140386.39512602412|5.3162145050241506E8|2.8669501991685364|
| stddev| 80562.27771851176| 3.929768082443173E8| 3.854183859201656|
|    min|                 2|                   0|               0.0|
|    max|            278854|          2130530508|              10.0|
+-------+------------------+--------------------+------------------+



In [16]:
from pyspark.sql.functions import isnan, when, count, col

df2.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+-------+----+-----------+
|User-ID|ISBN|Book-Rating|
+-------+----+-----------+
|      0|   0|          0|
+-------+----+-----------+



<h1 style="text-align:center">Runnning SQL queries Programmatically</h1>

In [17]:
df.createOrReplaceTempView("Book")

In [18]:
sqlDF=spark.sql("select `User-ID`,count(*) as Frequency from Book group by `User-ID`")

In [19]:
sqlDF.show()

+-------+---------+
|User-ID|Frequency|
+-------+---------+
| 277594|        1|
| 277840|        1|
| 278220|        2|
| 278659|        1|
|   1436|       12|
|   2136|       10|
|   3959|        2|
|   4032|        2|
|   4821|        3|
|   4937|        5|
|   5325|        1|
|   5925|        1|
|   6613|        1|
|   6731|       26|
|   7711|        1|
|   8433|        1|
|   9030|        1|
|   9583|        1|
|   9586|        2|
|   9993|        1|
+-------+---------+
only showing top 20 rows



In [20]:
df2=df2.dropna()


<h1 style="color:red;text-align:center;">Simple Algorithm for Recommendation (SAR)</h1>

In [21]:
from pyspark.ml.recommendation import ALS

In [22]:
# dataset split into training and testing set
(training, test) = df2.randomSplit([0.8, 0.2])
# training the model
als = ALS(maxIter=5, implicitPrefs=True,userCol="User-ID", itemCol="ISBN", ratingCol="Book-Rating",coldStartStrategy="drop")
model = als.fit(training)
# predict using the testing datatset
predictions = model.transform(test)
predictions.show()

+-------+--------+-----------+-------------+
|User-ID|    ISBN|Book-Rating|   prediction|
+-------+--------+-----------+-------------+
| 238849| 6251380|        0.0|-0.0013460224|
|  62398| 6722253|        0.0|          0.0|
| 206979| 6722253|        0.0|  9.628192E-5|
| 213255| 6724159|        0.0|    0.0168763|
|  11676|15232657|        8.0|  0.017755082|
| 218976|20427115|        5.0| 0.0080627855|
| 103630|20427115|        0.0|-0.0016064765|
| 187474|28604458|        7.0|          0.0|
| 250729|28604458|        0.0|          0.0|
| 167759|30080037|        5.0|          0.0|
| 160541|34071252|        0.0|  3.930893E-4|
|  17190|34543448|        0.0| -0.023150623|
| 118848|34543448|        0.0|-0.0044455705|
|  32440|34543448|        0.0| -0.014391618|
| 104939|34543448|        0.0|   0.01275183|
| 172742|34543448|       10.0|   0.06678126|
|  70594|34543448|        0.0|   0.06193844|
| 225810|34543448|        0.0|  0.036439203|
| 159376|34543448|        0.0| 0.0017757429|
| 227447|3

In [29]:

test.first()

Row(User-ID=2, ISBN=195153448, Book-Rating=0.0)

In [118]:
df2.count()

1062289

In [119]:
training.show()


+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
|      2| 195153448|          0|
|      7|  34542252|          0|
|      8|   2005018|          5|
|      8|  74322678|          5|
|      8|  80652121|          0|
|      8| 374157065|          0|
|      8| 393045218|          0|
|      8| 425176428|          0|
|      8| 679425608|          0|
|      8| 887841740|          5|
|      8|1552041778|          5|
|      8|1558746218|          0|
|      8|1567407781|          6|
|      8|1575663937|          6|
|      8|1881320189|          7|
|      9| 440234743|          0|
|     10|1841721522|          0|
|     12|1879384493|         10|
|     14|  61076031|          5|
|     14| 439095026|          5|
+-------+----------+-----------+
only showing top 20 rows



In [120]:
sc.setCheckpointDir('checkpoint/')
ALS.checkpointInterval = 2

In [ ]:
df_2=sc.textFile("rating.csv")

In [5]:
spark = SQLContext(sc)                                                                             
df = spark.createDataFrame([(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],["id", "category"])

In [8]:
stringIndexer = StringIndexer(inputCol="category", outputCol="categoryIndex")

In [9]:
model = stringIndexer.fit(df)

In [10]:
indexed = model.transform(df)

In [11]:
indexed.show()

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+

